In [670]:
# Import libraries
from msc_code.scripts.notebook_setup import *

In [671]:
import_path = os.path.join(PROC_DATA_DIR, 'data_cleaning', 'case_data_clean.csv')
case_data = pd.read_csv(import_path)

In [672]:
# CASE REPORTS

In [673]:
# Import case_data_clean to get Study_IDs for evaluation.
import_path = os.path.join(PROC_DATA_DIR, 'data_cleaning', 'case_data_clean.csv')
case_data_clean = pd.read_csv(import_path)

# Extract Study_IDs, store as list and sort ascending.
included_case_ids = sorted(list(case_data_clean['Study_ID'].unique()))
# Extract list of cases


In [674]:
case_data_clean.head()

,Study_ID,Patient_ID,Age_Yrs,Gender,Is_Intentional,Is_Prisoner,Is_Psych_Inpat,Is_Displaced_Person,Under_Influence_Alcohol,Psych_Hx,...,Object_Other_Long,Outcome_Endoscopy,Outcome_Surgery,Outcome_Death,Outcome_Injury_Needing_Intervention,Outcome_Perforation,Outcome_Obstruction,Outcome_Other,Outcome_Other_Long,Comments
0,39,39-001,31.0,Male,1.0,0.0,1.0,NaN,0.0,0.0,...,"5 bells, 1 battery",1.0,0.0,0.0,0.0,0.0,0.0,1.0,"""Fluroscopy""","""31 year old severely mentally and physically ..."
1,51,51-001,20.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Gastrointestinal Crosses,0.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN
2,51,51-002,20.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Gastrointestinal Crosses,0.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN
3,51,51-003,19.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Gastrointestinal Crosses,0.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN
4,51,51-004,21.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Gastrointestinal Crosses,0.0,1.0,0.0,1.0,1.0,0.0,0.0,NaN,NaN


In [675]:
# Create dataframe for case_bias_data, using JBI Check List for Case Reports

# 1. Were patient’s demographic characteristics clearly described?
# 2. Was the patient’s history clearly described and presented as a timeline?
# 3. Was the current clinical condition of the patient on presentation clearly described?
# 4. Were diagnostic tests or assessment methods and the results clearly described?
# 5. Was the intervention(s) or treatment procedure(s) clearly described?
# 6. Was the post-intervention clinical condition clearly described?
# 7. Were adverse events (harms) or unanticipated events identified and described?
# 8. Does the case report provide takeaway lessons?

col_long_dict = {
    1: "Were patient’s demographic characteristics clearly described?",
    2: "Was the patient’s history clearly described and presented as a timeline?",
    3: "Was the current clinical condition of the patient on presentation clearly described?",
    4: "Were diagnostic tests or assessment methods and the results clearly described?",
    5: "Was the intervention(s) or treatment procedure(s) clearly described?",
    6: "Was the post-intervention clinical condition clearly described?",
    7: "Were adverse events (harms) or unanticipated events identified and described?",
    8: "Does the case report provide takeaway lessons?"
}

col_short_dict = {
    1: "Patient_Demographic_Described",
    2: "History_Timeline",
    3: "Current_Condition_Described",
    4: "Diagnostic_Assessment_Described",
    5: "Intervention_Treatment_Described",
    6: "Post_Intervention_Condition_Described",
    7: "Harms_Described",
    8: "Takeaway_Lessons_Provided"
}

columns = ["Study_ID", "Patient_ID"] + list(col_short_dict.values()) + ["Overall_Appraisal", "Reviewer_Initials", "Review_Date", "Comments_Reasoning"]

case_report_bias_data = pd.DataFrame(columns=columns)

case_report_bias_data['Study_ID'] = case_data_clean['Study_ID']
case_report_bias_data['Patient_ID'] = case_data_clean['Patient_ID']

In [676]:
case_report_bias_data.head()
len(case_report_bias_data)

195

In [677]:
# Export dataframe to CSV for bias assessment
export_path = os.path.join(PROC_DATA_DIR, 'bias_assessment', 'case_report_bias_data_start.csv')
case_report_bias_data.to_csv(export_path, index=False)

In [678]:
# Define acceptable input parameters, as defined by JBI.
acceptable_params_dict = {
    'Y': 'Yes',
    'N': 'No',
    'U': 'Unclear',
    'NA': 'Not Applicable'
}

In [679]:
# CASE SERIES

In [680]:
# Define JBI case series check list columns
col_long_dict = {
    1: "Were there clear criteria for inclusion in the case series?",
    2: "Was the condition measured in a standard, reliable way for all participants included in the case series?",
    3: "Were valid methods used for identification of the condition for all participants included in the case series?",
    4: "Did the case series have consecutive inclusion of participants?",
    5: "Did the case series have complete inclusion of participants?",
    6: "Was there clear reporting of the demographics of the participants in the study?",
    7: "Was there clear reporting of clinical information of the participants?",
    8: "Were the outcomes or follow up results of cases clearly reported?",
    9: "Was there clear reporting of the presenting site(s)/clinic(s) demographic information?",
    10: "Was statistical analysis appropriate?"
}

# Create dictionary for use as column headers, from JBI case series checklist.
col_short_dict = {
    1: "Clear_Inclusion_Criteria",
    2: "Standard_Condition_Measurement",
    3: "Valid_ID_Method",
    4: "Consecutive_Inclusion",
    5: "Complete_Inclusion",
    6: "Clear_Demographic_Reporting",
    7: "Clear_Clinical_Info_Reporting",
    8: "Clear_Outcome_Followup_Reported",
    9: "Clear_Site_Demographic_Reporting",
    10: "Appropriate_Statistical_Analysis"
}

# Create column headers
columns = ['Study_ID'] + list(col_short_dict.values()) + ["Overall_Appraisal", "Reviewer_Initials", "Review_Date", "Comments_Reasoning"]

# Create dataframe
case_series_bias_data = pd.DataFrame(columns=columns)

# Export
export_path = os.path.join(PROC_DATA_DIR, 'bias_assessment', 'case_series_bias_data_start.csv')
case_series_bias_data.to_csv(export_path, index=False)

In [681]:
# TODO Mark cases without motivation as 'N' for column 1, could exclude these at this stage.
exclusion_criteria

{1: 'Full text not available in English.',
 2: 'Studies not focusing on intentional self-ingestion (into the gastrointestinal tract) of foreign object via the oral cavity (mouth) or where unclear if ingested.',
 3: 'Studies focussing solely on accidental ingestion.',
 4: 'Non-Human/ animal studies.',
 5: 'Reviews, editorials, commentaries, and opinion pieces without original empirical data.',
 6: 'Duplicate publications or studies with overlapping data sets (the most comprehensive or recent study will be included).',
 7: 'Studies focusing on ingestion or co-ingestion of substances (e.g. poisons, medications) rather than physical foreign objects.',
 8: 'Ingestions undertaken in controlled environment as part of voluntary study.',
 9: 'Ingestions not explicitly stated to be intentional and history not suggestive of deliberate ingestion (i.e. Age < 8, no history of previous ingestions, no psychiatric co-morbidities, not a prisoner/detainee/vulnerable group).',
 10: 'Does not meet inclusio

In [682]:
# Import jbi_case_report_data 
import_path = os.path.join(RAW_DATA_DIR, 'bias_assessment', 'case_report_bias_data_end.xlsx')
jbi_case_report_bias_data = pd.read_excel(import_path)
jbi_case_report_bias_data.head()

,Study_ID,Patient_ID,Patient_Demographic_Described,History_Timeline,Current_Condition_Described,Diagnostic_Assessment_Described,Intervention_Treatment_Described,Post_Intervention_Condition_Described,Harms_Described,Takeaway_Lessons_Provided,Overall_Appraisal,Reviewer_Initials,Review_Date,Comments_Reasoning
0,39,39-001,Y,Y,Y,Y,Y,Y,NaN,Y,Include,JGE,2025-05-08,"Detailed pertinent history, but lacks specific..."
1,51,51-001,Y,Y,Y,Y,Y,Y,Y,Y,Include,JGE,2025-05-09,No comment.
2,51,51-002,Y,Y,Y,Y,Y,Y,Y,Y,Include,JGE,2025-05-10,No comment.
3,51,51-003,Y,Y,Y,Y,Y,Y,Y,Y,Include,JGE,2025-05-11,No comment.
4,51,51-004,Y,Y,Y,Y,Y,Y,Y,Y,Include,JGE,2025-05-12,No comment.


In [683]:
jbi_case_report_bias_data.shape

(195, 14)

In [684]:
jbi_case_report_bias_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 14 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Study_ID                               195 non-null    int64         
 1   Patient_ID                             195 non-null    object        
 2   Patient_Demographic_Described          195 non-null    object        
 3   History_Timeline                       195 non-null    object        
 4   Current_Condition_Described            195 non-null    object        
 5   Diagnostic_Assessment_Described        195 non-null    object        
 6   Intervention_Treatment_Described       195 non-null    object        
 7   Post_Intervention_Condition_Described  195 non-null    object        
 8   Harms_Described                        117 non-null    object        
 9   Takeaway_Lessons_Provided              195 non-null    object    

In [685]:
data_cols = ['Study_ID', 'Patient_ID']
assessment_cols = ['Patient_Demographic_Described', 'History_Timeline', 'Current_Condition_Described', 
                   'Diagnostic_Assessment_Described',	'Intervention_Treatment_Described',	
                   'Post_Intervention_Condition_Described', 'Harms_Described']

In [686]:
# Replace np.nan values with 'NA' to record where 'Harms_Described' is not applicable.
jbi_case_report_bias_data['Harms_Described'] = jbi_case_report_bias_data['Harms_Described'].replace(np.nan, 'NA')

In [687]:
for col in jbi_case_report_bias_data[assessment_cols]:
    print(f"{col}: {jbi_case_report_bias_data[col].unique()}")

Patient_Demographic_Described: ['Y' 'N']
History_Timeline: ['Y' 'N']
Current_Condition_Described: ['Y' 'N']
Diagnostic_Assessment_Described: ['Y' 'N']
Intervention_Treatment_Described: ['Y' 'N']
Post_Intervention_Condition_Described: ['Y' 'N']
Harms_Described: ['NA' 'Y' 'N']


In [688]:
jbi_case_report_bias_data['Overall_Appraisal'].unique()

array(['Include', 'Exclude', 'Unclear'], dtype=object)

In [689]:
# Merge jbi_case_report_bias_data with case_data_clean
case_data_bias_data_merged = pd.merge(case_data_clean,
                           jbi_case_report_bias_data,
                           on=['Study_ID', 'Patient_ID'],
                           how='left')

case_data_bias_data_merged.columns

Index(['Study_ID', 'Patient_ID', 'Age_Yrs', 'Gender', 'Is_Intentional',
       'Is_Prisoner', 'Is_Psych_Inpat', 'Is_Displaced_Person',
       'Under_Influence_Alcohol', 'Psych_Hx', 'Severe_Disability_Hx',
       'Previous_Ingestions', 'Motivation_Intent_To_Harm',
       'Motivation_Protest', 'Motivation_Psychiatric',
       'Motivation_Psychosocial', 'Motivation_Unknown', 'Motivation_Other',
       'Motivation_Other_Long', 'Object_Button_Battery', 'Object_Magnet',
       'Object_Long', 'Object_Diameter_Large', 'Object_Sharp',
       'Object_Multiple', 'Object_Other_Long', 'Outcome_Endoscopy',
       'Outcome_Surgery', 'Outcome_Death',
       'Outcome_Injury_Needing_Intervention', 'Outcome_Perforation',
       'Outcome_Obstruction', 'Outcome_Other', 'Outcome_Other_Long',
       'Comments', 'Patient_Demographic_Described', 'History_Timeline',
       'Current_Condition_Described', 'Diagnostic_Assessment_Described',
       'Intervention_Treatment_Described',
       'Post_Intervention_Condi

In [690]:
# define outcome_cols
outcome_cols = [col for col in case_data_bias_data_merged.columns if col.startswith('Outcome_') and col != 'Outcome_Other_Long']

# Create Outcome_Unknown == Y column if outcome columns are all marked as 'UK'
case_data_bias_data_merged['Outcome_Unknown'] = case_data_bias_data_merged[outcome_cols] \
    .apply(lambda row: 'Y' if (row == 'UK').all() else 'N', axis=1)

# show values
case_data_bias_data_merged['Outcome_Unknown'].value_counts()

Outcome_Unknown
N    195
Name: count, dtype: int64

In [691]:
# define object_cols
object_cols = [col for col in case_data_bias_data_merged.columns if col.startswith('Object_') and col != 'Object_Other_Long']

# Create Outcome_Unknown == Y column if outcome columns are all marked as 'UK'
case_data_bias_data_merged['Object_Unknown'] = case_data_bias_data_merged[object_cols] \
    .apply(lambda row: 'Y' if (row == 'UK').all() else 'N', axis=1)

# show values
case_data_bias_data_merged['Object_Unknown'].value_counts()

Object_Unknown
N    195
Name: count, dtype: int64

In [692]:
# See where motivation is unknown
case_data_bias_data_merged['Motivation_Unknown'].value_counts()

Motivation_Unknown
1.0    102
0.0     89
Name: count, dtype: int64

In [693]:
case_data_motivation_known = case_data_bias_data_merged[case_data_bias_data_merged['Motivation_Unknown'] == 0]
print(f"Cases where motivation known: {len(case_data_motivation_known)}")

Cases where motivation known: 89


In [694]:
case_data_motivation_known['Overall_Appraisal'].value_counts()

Overall_Appraisal
Include    79
Exclude    10
Name: count, dtype: int64

In [695]:
# Remove cases marked as exclude
excluded_cases = case_data_motivation_known[case_data_motivation_known['Overall_Appraisal'] == 'Exclude']
print(f"Cases excluded during bias assessment: {len(excluded_cases)}")
case_data_motivation_known = case_data_motivation_known[case_data_motivation_known['Overall_Appraisal'] == 'Include']
print(f"Cases remaining for analysis {len(case_data_motivation_known)}")

Cases excluded during bias assessment: 10
Cases remaining for analysis 79


In [696]:
# export case_data_motivation_known
export_path = os.path.join(PROC_DATA_DIR, 'bias_assessment', 'case_data_with_bias_motivation_known.csv')
case_data_motivation_known.to_csv(export_path, index=False)

In [697]:
case_data_motivation_known

,Study_ID,Patient_ID,Age_Yrs,Gender,Is_Intentional,Is_Prisoner,Is_Psych_Inpat,Is_Displaced_Person,Under_Influence_Alcohol,Psych_Hx,...,Intervention_Treatment_Described,Post_Intervention_Condition_Described,Harms_Described,Takeaway_Lessons_Provided,Overall_Appraisal,Reviewer_Initials,Review_Date,Comments_Reasoning,Outcome_Unknown,Object_Unknown
1,51,51-001,20.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Y,Y,Y,Y,Include,JGE,2025-05-09,No comment.,N,N
2,51,51-002,20.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Y,Y,Y,Y,Include,JGE,2025-05-10,No comment.,N,N
3,51,51-003,19.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Y,Y,Y,Y,Include,JGE,2025-05-11,No comment.,N,N
4,51,51-004,21.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Y,Y,Y,Y,Include,JGE,2025-05-12,No comment.,N,N
5,51,51-005,29.0,Male,1.0,1.0,0.0,NaN,0.0,NaN,...,Y,Y,Y,Y,Include,JGE,2025-05-13,No comment.,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,661,661-001,44.0,Male,1.0,0.0,0.0,NaN,NaN,0.0,...,Y,Y,Y,Y,Include,JGE,2025-05-13,No comment.,N,N
188,667,667-001,91.0,Female,1.0,0.0,NaN,NaN,NaN,1.0,...,Y,Y,NA,Y,Include,JGE,2025-05-13,No comment.,N,N
190,680,680-001,22.0,Male,1.0,NaN,NaN,NaN,NaN,1.0,...,Y,Y,NA,Y,Include,JGE,2025-05-13,No comment.,N,N
193,686,686-001,39.0,Male,1.0,0.0,0.0,NaN,NaN,1.0,...,Y,Y,NA,Y,Include,JGE,2025-05-13,No comment.,N,N


In [698]:
case_data = case_data_motivation_known.copy()

In [699]:
case_data.columns

Index(['Study_ID', 'Patient_ID', 'Age_Yrs', 'Gender', 'Is_Intentional',
       'Is_Prisoner', 'Is_Psych_Inpat', 'Is_Displaced_Person',
       'Under_Influence_Alcohol', 'Psych_Hx', 'Severe_Disability_Hx',
       'Previous_Ingestions', 'Motivation_Intent_To_Harm',
       'Motivation_Protest', 'Motivation_Psychiatric',
       'Motivation_Psychosocial', 'Motivation_Unknown', 'Motivation_Other',
       'Motivation_Other_Long', 'Object_Button_Battery', 'Object_Magnet',
       'Object_Long', 'Object_Diameter_Large', 'Object_Sharp',
       'Object_Multiple', 'Object_Other_Long', 'Outcome_Endoscopy',
       'Outcome_Surgery', 'Outcome_Death',
       'Outcome_Injury_Needing_Intervention', 'Outcome_Perforation',
       'Outcome_Obstruction', 'Outcome_Other', 'Outcome_Other_Long',
       'Comments', 'Patient_Demographic_Described', 'History_Timeline',
       'Current_Condition_Described', 'Diagnostic_Assessment_Described',
       'Intervention_Treatment_Described',
       'Post_Intervention_Condi

In [700]:
bias_binary_cols = ['Patient_Demographic_Described', 'History_Timeline',
       'Current_Condition_Described', 'Diagnostic_Assessment_Described',
       'Intervention_Treatment_Described',
       'Post_Intervention_Condition_Described', 'Harms_Described',
       'Takeaway_Lessons_Provided']

demographic_cols = ['Is_Intentional',
       'Is_Prisoner', 'Is_Psych_Inpat', 'Is_Displaced_Person',
       'Under_Influence_Alcohol', 'Psych_Hx', 'Severe_Disability_Hx',
       'Previous_Ingestions']


In [701]:
# Set 'Patient_Demographic_Described' to 'U' for Unclear 
case_data.loc[
    (case_data[demographic_cols] == 'UK').any(axis=1),
    'Patient_Demographic_Described'
] = 'U'

# View results
case_data['Patient_Demographic_Described'].value_counts()

Patient_Demographic_Described
Y    73
N     6
Name: count, dtype: int64

In [702]:
export_path = os.path.join(PROC_DATA_DIR, 'bias_assessment', 'case_data.csv')
case_data.to_csv(export_path, index=False)


In [703]:
bias_binary_cols

['Patient_Demographic_Described',
 'History_Timeline',
 'Current_Condition_Described',
 'Diagnostic_Assessment_Described',
 'Intervention_Treatment_Described',
 'Post_Intervention_Condition_Described',
 'Harms_Described',
 'Takeaway_Lessons_Provided']

In [704]:
case_data['Patient_Demographic_Described'].value_counts(normalize=True)

Patient_Demographic_Described
Y    0.924051
N    0.075949
Name: proportion, dtype: float64

In [705]:
case_data['History_Timeline'].value_counts()

History_Timeline
Y    75
N     4
Name: count, dtype: int64

## Series Data

In [706]:
# Import series_data 
import_path = os.path.join(PROC_DATA_DIR, 'data_cleaning', 'series_data_clean.csv')
series_data = pd.read_csv(import_path)
print(f"Case Series with Data Available: {len(series_data)}")

Case Series with Data Available: 31


In [707]:
# Remove series where Motivation_Unknown_Rate == 1
series_data_motivation_unknown = series_data[series_data['Motivation_Unknown_Rate'] == 1]
print(f"Series where motivation is Unknown: {len(series_data_motivation_unknown)}")
series_data = series_data[series_data['Motivation_Unknown_Rate'] != 1]
print(f"Series still included: {len(series_data)}")

Series where motivation is Unknown: 22
Series still included: 9


In [708]:
# Remove series where Outcome is Unknown

outcome_cols = [col for col in series_data.columns if col.startswith('Outcome_') and col != 'Outcome_Other_Long']

# Create Outcome_Unknown col

In [709]:
# Create Outcome_Unknown == 1 column if outcome columns are all marked as nan
series_data['Outcome_Unknown_Rate'] = series_data[outcome_cols] \
    .apply(lambda row: 1 if (row.isna()).all() else 0, axis=1)

series_data['Outcome_Unknown_Rate'].value_counts()

series_data_outcome_unknown = series_data[series_data['Outcome_Unknown_Rate'] == 1]
print(f"Series remaining with Unknown Outcome: {len(series_data_outcome_unknown)}")

series_data = series_data[series_data['Outcome_Unknown_Rate'] == 0]
print(f"Series remaining: {len(series_data)}")

Series remaining with Unknown Outcome: 0
Series remaining: 9


In [710]:
# Remove series with Object Dimensions Unknown

object_cols = [col for col in series_data.columns if col.startswith('Object_') and col != 'Object_Count']

series_data['Object_Unknown_Rate'] = series_data[object_cols] \
    .apply(lambda row: 1 if (row.isna()).all() else 0, axis=1)

series_data_object_unknown = series_data[series_data['Object_Unknown_Rate'] == 1]

series_data = series_data[series_data['Object_Unknown_Rate'] != 1]

series_data['Object_Unknown_Rate'].value_counts()

Object_Unknown_Rate
0    7
Name: count, dtype: int64

In [711]:
series_data_not_intentional = series_data[series_data['Is_Intentional_Rate'] != 1]
print(f"Series where all ingestions not intentional: {len(series_data_not_intentional)}")

series_data = series_data[series_data['Is_Intentional_Rate'] == 1]
print(f"Series Remaining: {len(series_data)}")

Series where all ingestions not intentional: 4
Series Remaining: 3


In [712]:
series_data

,Study_ID,Comments,Patient_Count,Case_Count,Age_Low,Age_High,Age_Mean,Age_Median,Verified_Ingestion_Cases,Verified_Ingestion_Rate,...,Outcome_Obstruction_Rate,Outcome_Other_Cases,Outcome_Other_Rate,Outcome_Other_Long,Outcome_Conservative_Cases,Outcome_Conservative_Rate,Outcome_Endoscopy_Surgery_Cases,Outcome_Endoscopy_Surgery_Rate,Outcome_Unknown_Rate,Object_Unknown_Rate
15,442,NaN,35.0,166.0,NaN,NaN,32.9,NaN,166.0,1.0,...,NaN,5.0,0.030120,NaN,NaN,NaN,NaN,NaN,0,0
20,523,"""The most common motive for swallowing was sui...",19.0,19.0,17.0,40.0,24.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,14.0,0.736842,NaN,NaN,0,0
26,646,NaN,16.0,19.0,19.0,27.0,24.0,NaN,NaN,1.0,...,0.052632,2.0,0.105263,pneumo-mediastinum in 2 cases,15.0,0.789474,1.0,0.052632,0,0


In [713]:
included_ids = list(case_data['Study_ID']) + list(series_data['Study_ID'])
print(f"Studies Included: {len(included_ids)}")

Studies Included: 82


In [717]:
# Import series bias data
import_path = os.path.join(RAW_DATA_DIR, 'bias_assessment', 'case_series_bias_data_end.xlsx')
series_bias_data = pd.read_excel(import_path)

In [719]:
series_bias_data.columns

Index(['Study_ID', 'Clear_Inclusion_Criteria',
       'Standard_Condition_Measurement', 'Valid_ID_Method',
       'Consecutive_Inclusion', 'Complete_Inclusion',
       'Clear_Demographic_Reporting', 'Clear_Clinical_Info_Reporting',
       'Clear_Outcome_Followup_Reported', 'Clear_Site_Demographic_Reporting',
       'Appropriate_Statistical_Analysis', 'Overall_Appraisal',
       'Reviewer_Initials', 'Review_Date', 'Comments_Reasoning',
       'Setting_Population', 'Study_Objective', 'Study_Method',
       'Study_Results', 'Study_Conclusion'],
      dtype='object')

In [ ]:
series_bias_cols = ['Clear_Inclusion_Criteria',
       'Standard_Condition_Measurement', 'Valid_ID_Method',
       'Consecutive_Inclusion', 'Complete_Inclusion',
       'Clear_Demographic_Reporting', 'Clear_Clinical_Info_Reporting',
       'Clear_Outcome_Followup_Reported', 'Clear_Site_Demographic_Reporting',
       'Appropriate_Statistical_Analysis']

Clear_Inclusion_Criteria  Standard_Condition_Measurement  Valid_ID_Method  Consecutive_Inclusion  Complete_Inclusion  Clear_Demographic_Reporting  Clear_Clinical_Info_Reporting  Clear_Outcome_Followup_Reported  Clear_Site_Demographic_Reporting  Appropriate_Statistical_Analysis
Y                         Y                               Y                Y                      Y                   Y                            Y                              Y                                Y                                 Y                                   3
Name: count, dtype: int64